In [2]:
from __future__ import print_function
from pathlib import Path
from random import shuffle
import random

import argparse
import copy
import numpy as np
import sys
import tensorflow as tf

from deeplp.models.data_prep import create_weighted_graph, load_data
from deeplp.models.data_prep import prepare_data, random_unlabel, calc_masks
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.models.utils import accuracy, indices_to_vec
from deeplp.models.data_prep import select_features

In [16]:
confidence=True

In [3]:
true_labels, features, edge_features, node_features, \
graph, weights = load_data('cora',directed=1,confidence=confidence)


-----------cora-----------
Loading labels...
Loading features...
Loading edge features...
Weights: Asymmetric
Loading graph...
Loading weights...
Done!


In [4]:

labeled_indices, unlabeled_indices = \
    random_unlabel(true_labels,0.99,features,
                   seed=0,confidence=confidence)


In [5]:

num_nodes, num_classes = true_labels.shape
labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices, logistic=0, confidence=confidence)

In [11]:
train_data, validation_data = prepare_data(labels,is_labeled,
                                           labeled_indices,None,true_labels,
                                           1,100,
                                           0)

In [17]:
lp = LP()
unlabeled_pred = lp.iter_sp(labels,
                     weights>0,
                     is_labeled,
                     100,
                     unlabeled_indices)
if confidence:
    unlabeled_pred = unlabeled_pred[:,:-1]
y_pred = np.argmax(unlabeled_pred,axis=1)
y_true = np.argmax(true_labels[unlabeled_indices],axis=1)
accuracy = np.mean(y_pred == y_true)


In [20]:
labels.shape

(2485, 8)

In [ ]:
true_labels.shape

In [18]:
trained_model = DeepLP_Edge('edge',
                            log_name,
                            edge_features, # [:,6:] for edge features only
                            graph,
                            labels.shape[1],
                            unlabeled_indices,
                            confidence,
                            loss_class_mass=args.loss_class_mass,
                            regularize=reg,
                            regularize_type=args.regularize_type,
                            num_iter=args.num_iter,
                            clamp=args.clamp,
                            lr=args.lr,
                            seed=args.parameter_seed,
                            weight_normalization=args.weight_normalization)

0.640650406504065